In [1]:
%matplotlib inline
import sys
sys.path.insert(1,'./Utilities')
# notebook
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
import networkx as nx
import json
from math import sin, cos, sqrt, atan2, radians
from utils import *
from data import *
from metaNetBusinessSide import *
from foodWebGraphing import *
#make the graphs bigger
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24

In [14]:
from data import validatedEiggData
import requests
import pickle

def findType(groups,values,granularity=None):
    if granularity is None:
        return [groups,values]

    consumableGroup = groups.lower().split("|")
    consumableValues = values.lower().split("|")
    
    accessable = dict(zip(consumableGroup,consumableValues))
    if granularity in accessable:
        return accessable[granularity]
    else:
        return [groups,values]
    
def getSpeciesType(name, granularity=None):
    callToValidateName = requests.get('http://resolver.globalnames.org/name_resolvers.json?names='+name)
    try:
        jsonRes = callToValidateName.json()['data'][0]["results"][0]
        groups,values = jsonRes["classification_path_ranks"], jsonRes["classification_path"]
        return findType(groups,values,granularity)
    except:
        return ['','']

def groupDataByType(species,granularity):
    types = list(map(lambda x: getSpeciesType(x, granularity), species))
    return dict(zip(species,types))

# politeness function- avoid hitting servers manually every time
def buildEiggTaxonomicIndex():
    taxonomicIndex = {}
    speciesScientificNames = validatedEiggData()
    animals = set(speciesScientificNames["Scientific name"].values.tolist())
    total = 0
    for s in animals:
        taxonomicIndex[s] = getSpeciesType(s)
        total += 1
        if total % 100 == 0:
            print("Step: " + str(total))
    
    with open("taxonomicIndexEigg", "wb") as f:
        pickle.dump(taxonomicIndex,f)

    with open("taxonomicIndexEigg", "rb") as f:
        print(len(pickle.load(f)))

    return taxonomicIndexs

def getSpeciesTaxonomicCategorty(name,category):
    with open("taxonomicIndexEigg", "rb") as f:
        ctx = pickle.load(f)

    groups,values = ctx[name]
    groups = groups.lower().split("|") 
    # ['', 'superkingdom', 'kingdom', 'phylum', 'subphylum', '', '', '', '', '', '', '', '', '', '', '', 'order', 'family', 'subfamily', '', 'genus', 'species']
    values = values.lower().split("|")

    taxonomy = dict(zip(groups,values))

    if category in taxonomy:
        return taxonomy[category]

In [15]:
df = eiggRawDataa()

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
for index, row in df.iterrows():
    try:
        print(row['Family'], getTaxonomyForAnimal(row['Scientific name'])['family'])
    except:
        pass
